## 👨🏻‍💻 Stock Technical Analysis Framework

### 📥 Setups

Installation and Import of required packages

In [247]:
# !pip install -r requirements.txt

In [248]:
import yfinance as yf
import backtrader as bt

import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

Initialization of Analysis Parameters

In [ ]:
TICKER: str = 'AAPL'

START_DATE: str = '2019-01-01'
END_DATE: str = '2025-07-25'

# Organized lists of indicators for plotting
PRICES_IND: list[str] = ['sma', 'ema', 'dema', 'wma', 'hma', 'bb_upper', 'bb_middle', 'bb_lower', 'keltner_upper', 'keltner_lower', 'donchian_upper', 'donchian_middle', 'donchian_lower', 'vwap']
TREND_IND: list[str] = ['macd', 'macd_signal', 'macd_hist', 'adx', 'aroon_up', 'aroon_down', 'dpo', 'ichimoku_senkou_span_a', 'ichimoku_senkou_span_b', 'ichimoku_kijun_sen', 'ichimoku_tenkan_sen']
MOMENTUM_IND_1: list[str] = ['rsi', 'stoch_k', 'stoch_d', 'stoch_rsi', 'williams_r']
MOMENTUM_IND_2: list[str] = ['cci', 'mfi', 'roc', 'momentum', 'kst', 'uo']
VOLUME_IND_1: list[str] = ['obv', 'cmf', 'adline', 'force_index', 'vroc']
VOLUME_IND_2: list[str] = ['volosc', 'eom', 'nvi', 'pvi']
VOLATILITY_IND_1: list[str] = ['atr', 'natr', 'chaikin_vol', 'hist_vol', 'ui']
VOLATILITY_IND_2: list[str] = ['bb_wdith', 'variance']

### 🏗️ Data Acquisition

Function for fetching stock OHCLV data

In [250]:
def fetch_price_data(ticker: str = TICKER, start: str = START_DATE, end: str = END_DATE) -> pd.DataFrame:
    """
    Fetches financial data from Yahoo Finance API for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols (Default to AAPL)
        start_date (str): Start date in 'YYYY-MM-DD' format (Default to 2019-01-01)
        end_date (str): End date in 'YYYY-MM-DD' format (Default to 2025-07-25)

    Returns:
        dict[str, pd.DataFrame]: Dictionary of DataFrames with date-indexed adjusted OHCLV + Log Return data
    """
    
    data = yf.download(ticker, start=start, end=end, auto_adjust=True)
    data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

    data['LogReturn'] = np.log(data['Close'] / data['Close'].shift(1))
    data.dropna(inplace=True)

    return data

The indicators will be split into 4 catgories, volume, volatility, momentum, and trend. The lists of specific indicators are as follows.
- Volume: Gauges <ins>strength of price movement</ins>.

| Indicator                         |                                                                                       Description                                                                                                         |
|:---------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| Volume Rate of Change (VROC) | Rate at which volume changes over time. Useful for identifying increasing trading activity that may precede price moves. |
| On Balance Volume (OBV) | Cumulative volume that adds volume on up days and subtracts on down days. Useful for confirming price trends. |
| Volume Weighted Average Price (VWAP) | Average price weighted by volume throughout the day. Useful for judging fair value. |
| Negative Volume Index (NVI) | Running total of price changes on days with decreasing volume, assuming smart money trades on low volume. |
| Positive Volume Index (PVI) | Running total of price changes on days with increasing volume, assuming retail trading dominates high-volume days. |
| Volume Oscillator (VolumeOsc) | Difference between short and long-term volume moving averages. Useful for identifying volume surges or drying activity. |
| Ease of Movement (EoM) | Quotient between price distance moved and volume. Useful for assessing the strength of a trend (how easily price moves). |
| Force Index (ForceIndex) | Product of closing price difference and traving volume to measure buying/selling pressure. Positive——bullish; Negative——bearish. |
| Chaikin Money Flow (CMF)  | Quotient between sum of intraday price movements and sum of volumes to measure accumulation and dsitribution. Positive——bullish; Negative——bearish. |
| Accumulation/Distribution Line (ADLine) | Cumulative volume that multiplies by prive movement. Rising——bullish; Falling——bearish; Divergence——reversal |

---
- Volatility: Measures the <ins>degree of price variation</ins> over time.

| Indicator                         |                                                                                       Description                                                                                                         |
|:---------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| Historical Volatility (HistVolatility) | Standard deviation of price returns over a specified period. Higher values indicate more risk. |
| Chaikin Volatility (ChaikinVolatility) | Percentage change in spread between high and low prices. Useful for revealing momentum shifts. |
| Ulcer Index (UI) | Standard deviation of drawdowns to quantify downside risk. Useful for risk-aversing. |
| Variance Indicator (Variance) | Dispersion of price returns. |
| Average True Range (ATR) | Average price range of an investment over a period. |
| Normalized ATR (NATR) | ATR scaled to percentage of current price, allowing volatility comparison across assets. |
| Bollinger Bands (BollingerBands) | A moving average and bands at ±2 standard deviations. Useful for detecting overbought/oversold levels. |
| Width of Bollinger Bands (BBWidth) | Distance between upper and lower Bollinger Bands. High width——high volatility. |
| Donchian Channel (DonchianChannel) | Highest high and lowest low over a period. Useful for identifying breakouts. |
| Keltner Channel (KeltnerChannel) | A moving average and bands at ±2 ATR. Useful for assessing volatility. |

---
- Momentum: Measures the <ins>speed of price movement</ins>, useful for identifying overbought/oversold conditions.

| Indicator                         |                                                                                       Description                                                                                                         |
|:---------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| Relative Strength Index (RSI) | Ratio of average gain to average loss, scaled 0–100. 70——overbought； 30——oversold. |
| Commodity Channel Index (CCI) | Deviation of a price from a moving average. 100——overbought; -100——oversold. |
| Money Flow Index (MFI) | RSI with volume. Useful for identifying money flow into/out of asset. |
| Ultimate Oscillator (UltimateOscillator) | Buying pressure over ATR racross multiple timeframes to reduce false signals. |
| Stochastic Oscillator (Stochastic) | Quotient of difference between close and low and price range over a period. 80——overbought 20——oversold. |
| Stochastic RSI (StochRSI) | Quotient of difference between RSI and lowest RSI and RSI range over a period. Useful for pinpointing reversals. |
| Rate of Change (ROC) | % change in price between current and past periods. Useful for spotting inflection points. |
| KnowSureThing (KST) | Weighted sum of ROC values at different intervals. Useful for identifying cycles in price movement. |
| Williams %R (WilliamsR) | Quotient of difference between highest high and close and high-low range over a period, scaled 0–-100. -20——overbought -80——oversold. |
| Momentum Indicator (Momentum) | Velocity of price. Useful for identifying trend continuation or reversal. |

---
- Trend: Shows the <ins>direction and strength of price movement</ins>.

| Indicator                         |                                                                                       Description                                                                                                         |
|:---------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| Simple Moving Average (SMA) | Average closing price over a period. |
| Exponential Moving Average (EMA) | SMA but weights recent prices more. Faster to adapt to changes. |
| Double Exponential Moving Average (DEMA) | EMA of EMA to reduce lag. |
| Weighted Moving Average (WMA) | Weighted SMA where recent prices matter most. |
| Hull Moving Average (HMA) | WMA of weighted difference between short and long period WMA. Attempts to reduce lag while maintaining smoothness. |
| Moving Average Convergence Divergence (MACD) | EMAs of different periods to track momentum and trend. Includes MACD line, signal line, and histogram. |
| Detrended Price Oscillator (DPO) | SMA of half a period ago. Useful for identifying short-term cycles. |
| Average Directional Index (ADX) | Ratio of high and low price moves to measure trend strength, scaled to 0-100. 25——trend; 20——neutral. |
| Aroon Indicator (AroonUp/AroonDown) | Number of days since highest high or lowest low, scaled to 0-100. Extreme value——trend. |
| Ichimoku Cloud (Ichimoku) | Central price level over various periods stacked at different timeframes to show trend, support/resistance, and momentum in one view. Price above cloud——bullish; Price below cloud——bearish. |

Class for feeding data

In [251]:
class PandasData(bt.feeds.PandasData):
    """
    Custom Backtrader Data Feed that supports OHLCV + LogReturn column.
    """
    
    lines = ('logreturn',)
    params = (('datetime', None), ('open', 'Open'), ('high', 'High'), ('low', 'Low'), ('close', 'Close'), ('volume', 'Volume'), ('logreturn', 'LogReturn'), ('openinterest', None))  # Parameters to convert to Backtrader datafeed

Classes for some non-built-in technical indicators

In [252]:
class MFI(bt.Indicator):
    """
    Money Flow Index (MFI)

    Params:
        period (int): The number of periods to use in calculation (default: 14)

    Output:
        mfi (line): Money Flow Index values
    """
        
    lines = ('mfi',)
    params = dict(period=14)

    alias = ('MoneyFlowIndicator',)

    def __init__(self):
        tprice = (self.data.close + self.data.low + self.data.high) / 3.0
        mfraw = tprice * self.data.volume

        flowpos = bt.ind.SumN(mfraw * (tprice > tprice(-1)), period=self.p.period)
        flowneg = bt.ind.SumN(mfraw * (tprice < tprice(-1)), period=self.p.period)

        mfiratio = bt.ind.DivByZero(flowpos, flowneg, zero=100.0)
        self.l.mfi = 100.0 - 100.0 / (1.0 + mfiratio)

class DonchianChannels(bt.Indicator):
    """
    Donchian Channels

    Params:
        period (int): Lookback period (default: 20)
        lookback (int): -1 to exclude current bar, 0 to include current (default: -1)

    Output:
        dch (line): Upper band (max high)
        dcl (line): Lower band (min low)
        dcm (line): Midline (average of dch and dcl)
    """

    alias = ('DCH', 'DonchianChannel',)

    lines = ('dcm', 'dch', 'dcl',)  # dc middle, dc high, dc low
    params = dict(
        period=20,
        lookback=-1,  # consider current bar or not
    )

    plotinfo = dict(subplot=False)  # plot along with data
    plotlines = dict(
        dcm=dict(ls='--'),  # dashed line
        dch=dict(_samecolor=True),  # use same color as prev line (dcm)
        dcl=dict(_samecolor=True),  # use same color as prev line (dch)
    )

    def __init__(self):
        hi, lo = self.data.high, self.data.low
        if self.p.lookback:  # move backwards as needed
            hi, lo = hi(self.p.lookback), lo(self.p.lookback)

        self.l.dch = bt.ind.Highest(hi, period=self.p.period)
        self.l.dcl = bt.ind.Lowest(lo, period=self.p.period)
        self.l.dcm = (self.l.dch + self.l.dcl) / 2.0  # avg of the above

class Variance(bt.Indicator):
    """
    Rolling Variance

    Params:
        period (int): Lookback window for calculating the variance

    Output:
        variance (line): Rolling variance value
    """

    lines = ('variance',)
    params = (('period', 20),)

    def __init__(self):
        mean = bt.ind.SimpleMovingAverage(self.data, period=self.p.period)
        self.lines.variance = bt.ind.SumN((self.data - mean) ** 2, period=self.p.period) / self.p.period

class OBV(bt.Indicator):
    """
    On-Balance Volume (OBV)

    Output:
        obv (line): OBV value
    """
 
    lines = ('obv',)
    params = ()

    def __init__(self):
        self.addminperiod(1)

    def next(self):
        if len(self) == 1:
            self.lines.obv[0] = self.data.volume[0]  # Init with first volume
        else:
            if self.data.close[0] > self.data.close[-1]:
                self.lines.obv[0] = self.lines.obv[-1] + self.data.volume[0]
            elif self.data.close[0] < self.data.close[-1]:
                self.lines.obv[0] = self.lines.obv[-1] - self.data.volume[0]
            else:
                self.lines.obv[0] = self.lines.obv[-1]

class VWAP(bt.Indicator):
    """
    Volume Weighted Average Price (VWAP)

    Output:
        vwap (line): VWAP value
    """

    lines = ('vwap',)

    def __init__(self):
        typical_price = (self.data.high + self.data.low + self.data.close) / 3
        cum_pv = bt.ind.SumN(typical_price * self.data.volume)
        cum_vol = bt.ind.SumN(self.data.volume)
        self.lines.vwap = cum_pv / cum_vol

class VI(bt.Indicator):
    """
    Volume Index (Positive or Negative)

    Params:
        mode (str): 'pvi' for Positive Volume Index, 'nvi' for Negative Volume Index
        initial (float): Starting value of index (default: 1000)

    Output:
        vi (line): Volume Index
    """

    lines = ('vi',)
    params = (
        ('mode', 'pvi'),  # 'pvi' or 'nvi'
        ('initial', 1000.0),
    )

    def __init__(self):
        self.addminperiod(1)

    def next(self):
        if len(self) == 1:
            self.lines.vi[0] = self.p.initial
            return

        # Choose mode logic
        vol_up = self.data.volume[0] > self.data.volume[-1]
        vol_down = self.data.volume[0] < self.data.volume[-1]

        if (self.p.mode == 'nvi' and vol_down) or (self.p.mode == 'pvi' and vol_up):
            change = (self.data.close[0] - self.data.close[-1]) / self.data.close[-1]
            self.lines.vi[0] = self.lines.vi[-1] * (1 + change)
        else:
            self.lines.vi[0] = self.lines.vi[-1]

class EOM(bt.Indicator):
    """
    Ease of Movement (EoM)

    Output:
        eom (line): Smoothed Ease of Movement value
    """

    lines = ('eom',)
    params = dict(period=14, vol_scale_period=20)

    def __init__(self):
        self.addminperiod(max(self.p.period, self.p.vol_scale_period) + 1)

        # Calculate midpoints and price distance
        mid = (self.data.high + self.data.low) / 2
        mid_prev = mid(-1)
        distance = mid - mid_prev

        # Calculate box ratio with dynamic volume scaling
        spread = self.data.high - self.data.low + 1e-10  # avoid divide-by-zero
        vol_scale = bt.ind.SMA(self.data.volume, period=self.p.vol_scale_period)
        box_ratio = self.data.volume / (vol_scale * spread)

        # Raw EoM
        eom_raw = distance / box_ratio

        # Smoothed EoM
        self.lines.eom = bt.ind.SMA(eom_raw, period=self.p.period)

class FI(bt.Indicator):
    """
    Force Index (FI)

    Output:
        force (line): Force Index value
    """

    lines = ('force',)

    def __init__(self):
        self.addminperiod(2)
        self.lines.force = (self.data.close - self.data.close(-1)) * self.data.volume

In [253]:
class IndicatorStrategy(bt.Strategy):
    """
    A Backtrader strategy that computes and stores a wide range of technical indicators spanning trend, momentum, volatility, and volume dimensions.
    """

    def __init__(self):
        """
        Initializes the IndicatorStrategy by constructing a comprehensive suite of technical indicators.
        """
        self.full_data = []

        # Trend
        self.sma = bt.ind.SMA(self.data.close, period=20)
        self.ema = bt.ind.EMA(self.data.close, period=20)
        self.dema = bt.ind.DEMA(self.data.close, period=20)
        self.wma = bt.ind.WMA(self.data.close, period=20)
        self.hma = bt.ind.HullMovingAverage(self.data.close, period=16)
        self.macd = bt.ind.MACD(self.data.close)
        self.dpo = bt.ind.DPO(self.data.close)
        self.adx = bt.ind.ADX(self.data)
        self.aroon = bt.ind.AroonIndicator(self.data, period=14)
        self.ichimoku = bt.ind.Ichimoku()
        
        # Momentum
        self.rsi = bt.ind.RSI(self.data.close, period=14)
        self.cci = bt.ind.CCI(self.data, period=20)
        self.mfi = MFI(self.data, period=14)
        self.uo = bt.ind.UltimateOscillator(self.data)
        self.kst = bt.ind.KST(self.data.close)
        self.stoch = bt.ind.Stochastic(self.data)
        self.stoch_rsi = (self.rsi - bt.ind.Lowest(self.rsi, period=14)) / (bt.ind.Highest(self.rsi, period=14) - bt.ind.Lowest(self.rsi, period=14))
        self.roc = bt.ind.RateOfChange(self.data.close, period=14)
        self.williams_r = bt.ind.WilliamsR(self.data)
        self.momentum = bt.ind.Momentum(self.data.close, period=14)
        
        # Volatility
        ema_hl = bt.ind.EMA(self.data.high - self.data.low, period=10)
        drawdown = (self.data.close - bt.ind.Highest(self.data.close, period=14)) / bt.ind.Highest(self.data.close, period=14)

        self.hist_vol = bt.ind.StdDev(self.data.logreturn, period=20) * (252 ** 0.5)
        self.chaikin_vol = (ema_hl - ema_hl(-10)) / ema_hl(-10)
        self.ui = bt.ind.StdDev(drawdown, period=14)
        self.variance = Variance(self.data.close, period=20)
        self.atr = bt.ind.ATR(self.data, period=14)
        self.natr = (self.atr / self.data.close) * 100
        self.bbands = bt.ind.BollingerBands(self.data.close, period=20, devfactor=2)
        self.bbwidth = (self.bbands.top - self.bbands.bot) / self.bbands.mid * 100
        self.donchian = DonchianChannels(self.data, period=20)
        self.keltner_upper = self.ema + 2 * self.atr
        self.keltner_lower = self.ema - 2 * self.atr
        
        # Volume
        short_ema = bt.ind.EMA(self.data.volume, period=14)
        long_ema = bt.ind.EMA(self.data.volume, period=28)
        mfm = ((self.data.close - self.data.low) - (self.data.high - self.data.close)) / (self.data.high - self.data.low)

        self.vroc = bt.ind.RateOfChange(self.data.volume, period=14)
        self.obv = OBV(self.data)
        self.vwap = VWAP(self.data)
        self.nvi = VI(self.data, mode='nvi')
        self.pvi = VI(self.data, mode='pvi')
        self.volosc = 100 * (short_ema - long_ema) / long_ema
        self.eom = EOM(period=14)  # smoothed EoM
        self.force_index = FI(self.data)
        self.cmf = bt.ind.SumN(mfm * self.data.volume, period=21) / bt.ind.SumN(self.data.volume, period=21)
        self.adline = bt.indicators.Accum(bt.If(bt.And(self.data.high != self.data.low, self.data.volume != 0), mfm * self.data.volume, 0))
    
    def next(self):
        """
        Appends a snapshot of all current indicator values to `self.full_data` at each time step.
        """
        
        self.full_data.append({
            'datetime': self.data.datetime.datetime(0),

            'sma': self.sma[0],
            'ema': self.ema[0],
            'dema': self.dema[0],
            'wma': self.wma[0],
            'hma': self.hma[0],
            'macd': self.macd.macd[0],
            'macd_signal': self.macd.signal[0],
            'macd_hist': self.macd.macd[0]-self.macd.signal[0],
            'dpo': self.dpo[0],
            'adx': self.adx[0],
            'aroon_up': self.aroon.aroonup[0],
            'aroon_down': self.aroon.aroondown[0],
            'ichimoku_senkou_span_a': self.ichimoku.senkou_span_a[0],
            'ichimoku_senkou_span_b': self.ichimoku.senkou_span_b[0],
            'ichimoku_kijun_sen': self.ichimoku.kijun_sen[0],
            'ichimoku_tenkan_sen': self.ichimoku.tenkan_sen[0],

            'rsi': self.rsi[0],
            'cci': self.cci[0],
            'mfi': self.mfi[0],
            'uo': self.uo[0],
            'kst': self.kst[0],
            'stoch_k': self.stoch.percK[0],
            'stoch_d': self.stoch.percD[0],
            'stoch_rsi': self.stoch_rsi[0],
            'roc': self.roc[0],
            'williams_r': self.williams_r[0],
            'momentum': self.momentum[0],

            'hist_vol': self.hist_vol[0],
            'chaikin_vol': self.chaikin_vol[0],
            'ui': self.ui[0],
            'variance': self.variance[0],
            'atr': self.atr[0],
            'natr': self.natr[0],
            'bb_upper': self.bbands.top[0],
            'bb_middle': self.bbands.mid[0],
            'bb_lower': self.bbands.bot[0],
            'bb_width': self.bbwidth[0],
            'donchian_upper': self.donchian.dch[0],
            'donchian_lower': self.donchian.dcl[0],
            'donchian_middle': self.donchian.dcm[0],
            'keltner_upper': self.keltner_upper[0],
            'keltner_lower': self.keltner_lower[0],

            'vroc': self.vroc[0],
            'obv': self.obv[0],
            'vwap': self.vwap[0],
            'nvi': self.nvi[0],
            'pvi': self.pvi[0],
            'volosc': self.volosc[0],
            'eom': self.eom[0],
            'force_index': self.force_index[0],
            'cmf': self.cmf[0],
            'ad': self.adline[0]
        })

### 📈 Visualization

Function for plotting candlesticks and moving averages

In [ ]:
def plot_prices(df_prices: pd.DataFrame, df_indicators: pd.DataFrame, fig: go.Figure, row: int = 1 , col: int = 1) -> None:
    """
    Adds traces for closing price and all moving averages to a specific subplot.

    Parameters:
        df_prices (pd.DataFrame): DataFrame containing OHCLV data.
        df_indicators (pd.DataFrame): DataFrame containing moving average data.
        fig (go.Figure): Plotly Figure object to which MACD traces will be added.
        row (int): The row number of the subplot to add MACD-related plots to.
        col (int): The column number of the subplot to add MACD-related plots to.
    """

    # Candlestick
    fig.add_trace(go.Candlestick(x=df_prices.index, open=df_prices['Open'], high=df_prices['High'], low=df_prices['Low'], close=df_prices['Close'], name="Candlestick"), 
                  row=row, col=col)

    # Moving Averages
    ma_names = ['sma', 'ema', 'dema', 'wma', 'hma']
    for ma_name in ma_names:
        fig.add_trace(go.Scatter(
            x=df_indicators.index,
            y=df_indicators[ma_name],
            mode='lines',
            name=ma_name.upper(),
            visible='legendonly' if ma_name != 'ema' else True # Only EMA visible initially
        ), row=row, col=col)
    
    # VWAP
    fig.add_trace(go.Bar(x=df_prices.index, y=df_indicators['vwap'], name='VWAP'), row=row, col=col)


In [267]:
def plot_volume(df_prices: pd.DataFrame, df_indicators: pd.DataFrame, fig: go.Figure, row: int = 2 , col: int = 1) -> None:
    """
    Adds traces for volume and volume indicators to a specific subplot.

    Parameters:
        df_prices (pd.DataFrame): DataFrame containing OHCLV data.
        df_indicators (pd.DataFrame): DataFrame containing moving average data.
        fig (go.Figure): Plotly Figure object to which MACD traces will be added.
        row (int): The row number of the subplot to add MACD-related plots to.
        col (int): The column number of the subplot to add MACD-related plots to.
    """

    fig.add_trace(go.Bar(x=df_prices.index, y=df_prices['Volume'], name='Volume'), row=row, col=col)

    fig.add_trace(go.Scatter(x=df_indicators.index, y=df_indicators['obv'], mode='lines', name='OBV'), row=row, col=col)

In [268]:
def plot_rsi(df: pd.DataFrame, fig: go.Figure, row: int = 4, col: int = 1) -> None:
    """
    Adds line charts for MACD and MACD Signal, and a bar chart for MACD Histogram to a specific subplot row.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'rsi' column.
        fig (go.Figure): Plotly Figure object to which MACD traces will be added.
        row (int): The row number of the subplot to add MACD-related plots to.
        col (int): The column number of the subplot to add MACD-related plots to.
    """

    fig.add_trace(go.Scatter(x=df.index, y=df['rsi'], mode='lines', name='RSI', line=dict(color='gold')), row=row, col=col)
    fig.add_hline(y=70, line=dict(dash='dash', color='red'), row=row, col=col)
    fig.add_hline(y=30, line=dict(dash='dash', color='blue'), row=row, col=col)

In [269]:
def plot_macd(df: pd.DataFrame, fig: go.Figure, row: int = 3, col: int = 1) -> None:
    """
    Adds line charts for MACD and MACD Signal, and a bar chart for MACD Histogram to a specific subplot row.

    Parameters:
        df (pd.DataFrame): DataFrame containing 'macd', 'macd_signal', and 'macd_hist' columns.
        fig (go.Figure): Plotly Figure object to which MACD traces will be added.
        row (int): The row number of the subplot to add MACD-related plots to.
        col (int): The column number of the subplot to add MACD-related plots to.
    """

    fig.add_trace(
        go.Scattergl(x=df.index, y=df['macd'], name='MACD', line=dict(color='blue')),
        row=row, col=col
    )
    fig.add_trace(
        go.Scattergl(x=df.index, y=df['macd_signal'], name='MACD Signal', line=dict(color='orange')),
        row=row, col=col
    )
    fig.add_trace(
        go.Bar(x=df.index, y=df['macd_hist'], name='MACD Hist', marker_color='purple'),
        row=row, col=col
    )

In [270]:
d = fetch_price_data()
datafeed = PandasData(dataname=d)

cerebro = bt.Cerebro()
cerebro.addstrategy(IndicatorStrategy)
cerebro.adddata(datafeed)
results = cerebro.run()
strategy = results[0]

df = pd.DataFrame(strategy.full_data)
df.set_index('datetime', inplace=True)

[*********************100%***********************]  1 of 1 completed


In [276]:
# Setup figure with 8 rows (example: candlestick + 7 indicator rows)
fig = make_subplots(
    rows=8, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    row_heights=[0.3] + [0.1]*7,
    specs=[[{"type": "candlestick"}]] + [[{"type": "xy"}]]*7
)

fig.update_layout(
    title="AAPL Technical Indicators",
    template="plotly_white",
    height=800,
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=3, label="3m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(step="all")
            ]),
            bgcolor="black",
            font=dict(color='white')
        ),
        rangeslider=dict(visible=False),
        type="date"
    )
)

fig.show()